In [1]:
#Updated 21 JUL 2022

In [2]:
import pandas as pd
import numpy as np 
from sklearn.utils import shuffle 

In [3]:
res = {
 '010': 'თბილისი',
 '260': 'ლანჩხუთი',
 '610': 'ხელვაჩაური',
 '600': 'ქუთაისი',
 '310': 'მცხეთა',
 '380': 'საჩხერე',
 '470': 'ახალციხე',
 '360': 'საგარეჯო',
 '460': 'ჩოხატაური',
 '530': 'წყალტუბო',
 '190': 'ზუგდიდი',
 '420': 'ფოთი',
 '330': 'ოზურგეთი',
 '280': 'მარნეული',
 '350': 'რუსთავი',
 '120': 'გარდაბანი',
 '200': 'თელავი',
 '320': 'ნინოწმინდა',
 '590': 'გორი',
 '070': 'ახალქალაქი',
 '390': 'სენაკი',
 '130': 'გურჯაანი',
 '480': 'ჩხოროწყუ',
 '340': 'ონი',
 '430': 'ქარელი',
 '090': 'ბაღდათი',
 '240': 'კასპი',
 '180': 'ზესტაფონი',
 '450': 'ყვარელი',
 '030': 'ადიგენი',
 '210': 'თერჯოლა',
 '170': 'ვანი',
 '500': 'ცხინვალი',
 '160': 'დუშეთი',
 '140': 'დედოფლისწყარო',
 '660': 'ხულო',
 '540': 'ჭიათურა',
 '370': 'სამტრედია',
 '580': 'ხობი',
 '570': 'ხაშური',
 '250': 'ლაგოდეხი',
 '080': 'ახმეტა',
 '490': 'ცაგერი',
 '110': 'ბორჯომი',
 '440': 'ყაზბეგი',
 '100': 'ბოლნისი',
 '400': 'სიღნაღი',
 '300': 'მესტია',
 '050': 'ასპინძა',
 '040': 'ამბროლაური',
 '220': 'თეთრიწყარო',
 '410': 'ტყიბული',
 '230': 'თიანეთი',
 '550': 'ხონი',
 '270': 'ლენტეხი',
 '560': 'ხარაგაული',
 '290': 'მარტვილი',
 '150': 'დმანისი',
 '020': 'გორი',
 '510': 'წალენჯიხა',
 '520': 'წალკა',
 '060': 'ახალგორი',
 '003': 'თბილისი',
 '004': 'თბილისი',
 '005': 'თბილისი',
 '006': 'თბილისი',
 '007': 'თბილისი',
 '008': 'თბილისი',
 '009': 'თბილისი',
 '011': 'თბილისი',
 '012': 'თბილისი',
 '013': 'ამბროლაური',
 '014': 'ზუგდიდი',
 '015': 'თელავი',
 '016': 'მცხეთა',
 '017': 'ოზურგეთი',
 '018': 'რუსთავი',
 '019': 'ახალციხე',
 '021': 'ქუთაისი',
 '022': 'ბათუმი',
 '023': 'აფხაზეთი',
 '291': 'მარტვილი',
 '033': 'ადიგენი',
 '650': 'აფხაზეთი',
 '620': 'აფხაზეთი'}

In [4]:
def open_csv(csv):
    '''
    Purpose: Opens the CSV file and converts it into a dataframe.
    
    Input: Name of the CSV file
    '''
    df = pd.read_csv(csv)
    df = df.drop(df.columns[0],axis=1) #get rid of unwanted columns
    return df

In [5]:
#add born as column
def born_col(df):
    '''
    Purpose: To add the 'Born' column to the dataframe, indicating where the person was born. 
    the 'vaccinationPlace' column indicates where the winner was vacinnated,
    therefore we use the database of prefixes to indicate where the winner was born. 
    
    Input: dataframe of the webscraped CSV file obtained from the original tables
    '''
    born = []
    pid_pre = [i[:3] for i in df['personalId']] #contains the prefixes of all winner's personal ID numbers
     
    for prefix in pid_pre:
        flag = False 
        for key in res.keys():
            if(prefix == key):
                born.append(res[key])
                flag = True
        if(not flag): # in case the prefix is not found in the given prefix database
            born.append('blank')

    df['Born'] = born #append the locations indicated by the prefix to 'Born' column 
    return df

In [6]:
def split_shuffle(df):
    '''
    Purpose: Shuffles the main dataframe into 15 parts, since the lottery lasted for 15 weeks. 
    '''
    df = shuffle(df)
    df.reset_index(inplace=True,drop=True)
    df_list = []
    for x in range(0,17123,1223):
        if(x == 17122):
            df_list.append(df[17122:-1])
        df_list.append(df[x:x+1222])

    return df_list

In [7]:
def born_vacctype(df):
    '''
    Purpose: Counts the number of winners who were born in a specific location and got a specific vaccination. The following data is later used in Tables I and II.
    Returns: df_raw_vacctype = raw data
             df_DP_vacctype = differentially private data
    '''
    csv = 'lottoproject.csv'
    full_df = open_csv(csv)
    full_df = born_col(full_df)
    born_unique = full_df['Born'].unique()
    vacc_types = ['Born','Sinopharm','Pfizer','SINOVAC','AstraZeneca']

    lis_city = []
    lis_city_lap = []

    for born in born_unique:
    #Raw
        sino = int(len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'Sinopharm')]))
        pfi = int(len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'Pfizer')]))
        svac = int(len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'SINOVAC')]))
        az = int(len(df.loc[(df['Born'] == born) & (df['vaccineName'] == 'AstraZeneca')]))
        lis_city.append([born,sino,pfi,svac,az])

        #DP
        e = 0.5
    
        sino_dp = int(round(sino + np.random.laplace(0,1/e)))
        pfi_dp = int(round(pfi + np.random.laplace(0,1/e)))
        svac_dp = int(round(svac + np.random.laplace(0,1/e)))
        az_dp = int(round(az + np.random.laplace(0,1/e)))

        if(sino_dp < 0): sino_dp = 0
        if(pfi_dp < 0): pfi_dp = 0
        if(svac_dp < 0): svac_dp = 0
        if(az_dp < 0): az_dp = 0

        lis_city_lap.append([born,sino_dp,pfi_dp,svac_dp,az_dp])


    df_raw_vacctype = pd.DataFrame(columns=vacc_types)
    df_raw_vacctype = df_raw_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vacctype.columns))
    df_DP_vacctype = pd.DataFrame(columns=vacc_types)
    df_DP_vacctype = df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vacctype.columns))

    return df_raw_vacctype, df_DP_vacctype

In [8]:
def vaccplace_vacctype(df):
    '''
    Purpose: Counts the number of winners who were vaccinated in a specific location and got a specific vaccination. The following data is later used in Tables III and IV.
    Returns: df_raw_vacctype = raw data
             df_DP_vacctype = differentially private data
    '''
    csv = 'lottoproject.csv'
    fulldf = open_csv(csv)
    fulldf = born_col(fulldf)
    vaccplace_unique = fulldf['vaccinationPlace'].unique()
    vacc_types = ['vaccinationPlace','Sinopharm','Pfizer','SINOVAC','AstraZeneca']
    lis_city = []
    lis_city_lap = []

    for vaccplace in vaccplace_unique:
    #Raw
        #print(i)
        sino = int(len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'Sinopharm')]))
        pfi = int(len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'Pfizer')]))
        svac = int(len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'SINOVAC')]))
        az = int(len(df.loc[(df['vaccinationPlace'] == vaccplace) & (df['vaccineName'] == 'AstraZeneca')]))
        lis_city.append([vaccplace,sino,pfi,svac,az])

        #DP
        e = 0.5
    
        sino_dp = int(round(sino + np.random.laplace(0,1/e)))
        pfi_dp = int(round(pfi + np.random.laplace(0,1/e)))
        svac_dp = int(round(svac + np.random.laplace(0,1/e)))
        az_dp = int(round(az + np.random.laplace(0,1/e)))

        if(sino_dp < 0): sino_dp = 0
        if(pfi_dp < 0): pfi_dp = 0
        if(svac_dp < 0): svac_dp = 0
        if(az_dp < 0): az_dp = 0

        lis_city_lap.append([vaccplace,sino_dp,pfi_dp,svac_dp,az_dp])


    df_raw_vaccplace_vacctype = pd.DataFrame(columns=vacc_types)
    df_raw_vaccplace_vacctype = df_raw_vaccplace_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vaccplace_vacctype.columns))
    df_DP_vaccplace_vacctype = pd.DataFrame(columns=vacc_types)
    df_DP_vaccplace_vacctype = df_DP_vaccplace_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vaccplace_vacctype.columns))

    return df_raw_vaccplace_vacctype, df_DP_vaccplace_vacctype

In [17]:
csv = 'lottoproject.csv'
df = open_csv(csv)
df = born_col(df) # add Born column

u_born = df['Born'].unique()
u_vplace = df['vaccinationPlace'].unique()

born_cols = ['Born','Sinopharm','Pfizer','SINOVAC','AstraZeneca']
vplace_cols = ['vaccinationPlace','Sinopharm','Pfizer','SINOVAC','AstraZeneca']
vtypes = ['Sinopharm','Pfizer','SINOVAC','AstraZeneca']

#finalize raw datasets
#Born
df_raw_born_vacctype,x = born_vacctype(df)
#Vaccinationplace
df_raw_vaccplace_vacctype,y = vaccplace_vacctype(df)

#finalize DP datasets
#create empty datasets
df_dp_vaccplace_vacctype = pd.DataFrame(columns=vplace_cols)
df_dp_born_vacctype = pd.DataFrame(columns=born_cols)

#append 
df_dp_born_vacctype['Born'] = u_born
df_dp_vaccplace_vacctype['vaccinationPlace'] = u_vplace

df_dp_vaccplace_vacctype = df_dp_vaccplace_vacctype.fillna(0)
df_dp_born_vacctype = df_dp_born_vacctype.fillna(0)


C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\868758685.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vacctype = df_raw_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\868758685.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_DP_vacctype = df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\2378194575.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vaccplace_vacctype = df_raw_vaccplace_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vaccplace_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\237819457

In [10]:
#run 15 times (because lotto lasted 15 weeks) 
#P.S. takes about 3-4 minutes to run on my mac.
for i in range(15):
    df_list = split_shuffle(df) #split into 15 datasets
    #df_list = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15] 
    #configuring the combined datasets
    for dataframe in df_list:
        raw,dp_born_vacctype = born_vacctype(dataframe)
        raw,dp_vplace_vacctype = vaccplace_vacctype(dataframe) 

        df_dp_born_vacctype = df_dp_born_vacctype.add(dp_born_vacctype,fill_value=0)
        
        df_dp_vaccplace_vacctype = df_dp_vaccplace_vacctype.add(dp_vplace_vacctype,fill_value=0)   
         


C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\868758685.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vacctype = df_raw_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\868758685.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_DP_vacctype = df_DP_vacctype.append(pd.DataFrame(lis_city_lap,columns = df_DP_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\2378194575.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_raw_vaccplace_vacctype = df_raw_vaccplace_vacctype.append(pd.DataFrame(lis_city, columns = df_raw_vaccplace_vacctype.columns))
C:\Users\Giorgi\AppData\Local\Temp\ipykernel_14852\237819457

In [24]:
#finalize DP datasets
df_dp_born_vacctype['Born'] = u_born
df_dp_vaccplace_vacctype['vaccinationPlace'] = u_vplace

for col in vtypes:
    df_dp_vaccplace_vacctype[col] = df_dp_vaccplace_vacctype[col].floordiv(15)

for col in vtypes:
    df_dp_born_vacctype[col] = df_dp_born_vacctype[col].floordiv(15)



In [13]:
df_dp_born_vacctype.to_csv('dp_born.csv')

In [14]:
df_dp_vaccplace_vacctype.to_csv('dp_vaccplace.csv')

In [15]:
df_raw_born_vacctype.to_csv('raw_born.csv')

In [26]:
df_dp_born_vacctype.to_csv('dp_born_2.csv')